In [590]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

Next we need to import the kwik file to analyze spike

In [178]:
hdf5_file_name = '2017-04-22_11-41-43.kwik'
data_set_name = '/channel_groups/0/spikes/clusters/main/1/cluster_group' #where is located the data you want inside the hdf5 file tree

file    = h5py.File(hdf5_file_name, 'r')   # 'r' means that hdf5 file is open in read-only mode
#data = file[data_set_name]

# see file tree
def print_attrs(name, obj):
    print name
    for key, val in obj.attrs.iteritems():
        print "    %s: %s" % (key, val)

file.visititems(print_attrs)


application_data
application_data/klustakwik2
    num_cpus: 4
    num_starting_clusters: 200
application_data/spikedetekt
    threshold_strong_std_factor: 4.5
    use_single_threshold: 1
    filter_low: 500.0
    dtype: ['int16']
    threshold_weak_std_factor: 2.0
    n_channels: 32
    filter_lfp_low: 0
    raw_data_files: ['openephys.dat']
    chunk_size_seconds: 1
    filter_high_factor: 0.475
    prb_file: ['1x32_buzsaki']
    extract_s_after: 16
    detect_spikes: ['negative']
    n_excerpts: 50
    filter_butter_order: 3
    weight_power: 2
    excerpt_size_seconds: 1
    connected_component_join_size: 1
    extract_s_before: 16
    experiment_name: ['2017-04-22_11-41-43.kwik']
    sample_rate: 30000
    n_features_per_channel: 3
    chunk_overlap_seconds: 0.015
    pca_n_waveforms_max: 10000
    filter_lfp_high: 300
channel_groups
channel_groups/0
    channel_order: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31]
    adjacency_g

    cluster_group: 3
channel_groups/0/clusters/main/204/application_data
channel_groups/0/clusters/main/204/application_data/klustaviewa
    color: 2
channel_groups/0/clusters/main/205
    cluster_group: 3
channel_groups/0/clusters/main/205/application_data
channel_groups/0/clusters/main/205/application_data/klustaviewa
    color: 3
channel_groups/0/clusters/main/206
    cluster_group: 3
channel_groups/0/clusters/main/206/application_data
channel_groups/0/clusters/main/206/application_data/klustaviewa
    color: 8
channel_groups/0/clusters/main/207
    cluster_group: 3
channel_groups/0/clusters/main/207/application_data
channel_groups/0/clusters/main/207/application_data/klustaviewa
    color: 7
channel_groups/0/clusters/main/208
    cluster_group: 0
channel_groups/0/clusters/main/208/application_data
channel_groups/0/clusters/main/208/application_data/klustaviewa
    color: 3
channel_groups/0/clusters/main/209
    cluster_group: 0
channel_groups/0/clusters/main/209/application_data
ch

channel_groups/0/clusters/original/165/application_data
channel_groups/0/clusters/original/165/application_data/klustaviewa
    color: 9
channel_groups/0/clusters/original/166
    cluster_group: 3
channel_groups/0/clusters/original/166/application_data
channel_groups/0/clusters/original/166/application_data/klustaviewa
    color: 4
channel_groups/0/clusters/original/167
    cluster_group: 3
channel_groups/0/clusters/original/167/application_data
channel_groups/0/clusters/original/167/application_data/klustaviewa
    color: 5
channel_groups/0/clusters/original/168
    cluster_group: 3
channel_groups/0/clusters/original/168/application_data
channel_groups/0/clusters/original/168/application_data/klustaviewa
    color: 5
channel_groups/0/clusters/original/169
    cluster_group: 3
channel_groups/0/clusters/original/169/application_data
channel_groups/0/clusters/original/169/application_data/klustaviewa
    color: 3
channel_groups/0/clusters/original/17
    cluster_group: 3
channel_groups/0

In [424]:
auxGood = file['/channel_groups/0/clusters/main/']
#a.visititems(print_attrs)
#a.attrs['cluster_group']

c = 0;
tag = np.zeros(np.size(np.concatenate((range(1),range(2,np.size(auxGood)+1)))))
for cls in np.concatenate((range(1),range(2,np.size(auxGood)+1))):
    aux = file['/channel_groups/0/clusters/main/' + str(cls)]
    tag[c] = aux.attrs['cluster_group'] #cluster tag as described below for 1 and 2
    c = c+1
#1 is MUA (needs to be cleaned in a manual way, we'll have to build an interface to do it)
#2 is GOOD clusters, just extract and analyze

#/channel_groups/0/clusters/main/<cluster_number>/cluster_group
aux = np.where(tag >= 1); #need to sum 1 to retrieve correct clusters

GoodMUAclus = aux[0]+1;
GoodMUAclus

array([  4,   6,   8,  13,  16,  17,  18,  22,  26,  34,  49, 114, 116,
       117, 143, 192, 229, 230, 252, 261, 267])

In [480]:
auxCluster = file['/channel_groups/0/spikes/clusters/main']
#auxChannel.visititems(print_attrs)
# '/channel_groups/0/spikes/time_samples' address to get sample time for each spike
# '/channel_groups/0/spikes/clusters/main' addres to get cluster for each spike
# 
np.shape(auxCluster)
#auxChannel.attrs['features_masks']

(7632072,)

In [588]:
hdf5_file_name = '2017-04-22_11-41-43.kwx'
data_set_name = '/channel_groups/0/spikes/clusters/main/1/cluster_group' #where is located the data you want inside the hdf5 file tree

arquiv    = h5py.File(hdf5_file_name, 'r')   # 'r' means that hdf5 file is open in read-only mode
selectChannel = arquiv['/channel_groups/0/features_masks']
np.shape(selectChannel)
#selectChannel[23,:,1] #the second dimension is indexing channel first and features laters, as it was said in the google groups, but some exploring would be good


(7632072, 96, 2)

Separating to which channel a spike should be

In [587]:
#Changin arrays to numpy arrays to work better
auxCluster = np.array(auxCluster)
GoodMUAclus = np.array(GoodMUAclus)

#create a dictionare to save the spikes for each clsuter
from collections import defaultdict
Neurons = defaultdict(list)

for cls in range(np.size(GoodMUAclus)):
    Neurons[GoodMUAclus[cls]]
    
for idx in range(np.size(GoodMUAclus)):
        aux = np.where(auxCluster == GoodMUAclus[idx]) #selecting which spike is in good clusters
        Neurons[GoodMUAclus[idx]] = aux[0]
        
        #save in a python structure which firing goes to each clsuter
        #if Neurons[aux2[0]] == []:
        #    Neurons[aux2[0]] = np.array(idx)
        #else:
        #    Neurons[aux2[0]] = np.array([Neurons[aux2[0]],idx])
        
Neurons

defaultdict(list,
            {4: array([     11,      25,     163, ..., 7632031, 7632058, 7632063]),
             6: array([      3,       5,      10, ..., 7632038, 7632062, 7632070]),
             8: array([      9,      22,      26, ..., 7632066, 7632067, 7632071]),
             13: array([   1388,    1693,    3154, ..., 7631367, 7631803, 7631843]),
             16: array([     19,      24,      29, ..., 7632005, 7632010, 7632052]),
             17: array([     73,      75,     226, ..., 7632053, 7632059, 7632068]),
             18: array([     17,      28,      32, ..., 7631855, 7632013, 7632049]),
             22: array([     30,      45,      47, ..., 7631907, 7631911, 7631975]),
             26: array([    112,    5893,    6829, ..., 7631791, 7631795, 7631810]),
             34: array([    372,     734,     798, ..., 7631934, 7631943, 7632056]),
             49: array([    126,     146,     147, ..., 7626003, 7627329, 7628725]),
             114: array([    335,     359,     388

Now I should use Neurons dict to get timestamps and channel. I should change the key from dict to the channel value, not cluster number

In [589]:
auxTimeStamps = file['/channel_groups/0/spikes/time_samples'] #getting the time sample for the time stamps
NeuronTS = defaultdict(list)
AllTS = range(np.size(NeuronTS))
for cls in range(np.size(GoodMUAclus)):
    aux = Neurons[GoodMUAclus[cls]][:]
    auxArr = np.zeros(np.size(aux))#array auxiliar
    for idx in range(np.size(aux)):
        auxArr[idx] = auxTimeStamps[aux[idx]]
    NeuronTS[GoodMUAclus[cls]] = auxArr
    
NeuronTS

defaultdict(list,
            {4: array([  1.12100000e+03,   2.93500000e+03,   1.33480000e+04, ...,
                      4.38111731e+08,   4.38114593e+08,   4.38114926e+08]),
             6: array([  3.07000000e+02,   4.86000000e+02,   1.11200000e+03, ...,
                      4.38112794e+08,   4.38114834e+08,   4.38115290e+08]),
             8: array([  1.03900000e+03,   2.39600000e+03,   2.95100000e+03, ...,
                      4.38115129e+08,   4.38115134e+08,   4.38115296e+08]),
             13: array([  9.46210000e+04,   1.16348000e+05,   2.34077000e+05, ...,
                      4.38050598e+08,   4.38090424e+08,   4.38093788e+08]),
             16: array([  2.15200000e+03,   2.76200000e+03,   3.16900000e+03, ...,
                      4.38109119e+08,   4.38109623e+08,   4.38113851e+08]),
             17: array([  6.39400000e+03,   6.46200000e+03,   1.89000000e+04, ...,
                      4.38113922e+08,   4.38114623e+08,   4.38115189e+08]),
             18: array([  1.918

For next we should save the channel to which one pertains and save the dict to use for analysis

In [614]:
Channels = np.zeros(np.size(GoodMUAclus))
for cls in range(np.size(GoodMUAclus)):
    aux = Neurons[GoodMUAclus[cls]][:]
    auxArr = np.zeros(np.size(aux))#array auxiliar
    #temp = np.zeros(32)
    for idx in range(np.size(aux)):
        for ch in range(32):
            temp = np.sum(selectChannel[aux[idx],(3*ch):(3*ch)+3,1]) # three features per channel
            if (temp == 3):
                auxArr[idx] = ch
                break
    auxCh = stats.mode(auxArr)
    Channels[cls] = auxCh[0][0]
    
Channels
    
#selectChannel[spikenumber,feature,1]

array([ 24.,  27.,   0.,  22.,   9.,  28.,  13.,  10.,  20.,   0.,  29.,
         0.,  26.,  22.,   1.,  14.,  20.,  28.,  28.,  31.,  29.])

In [637]:
NeuronTS['Channels'] = Channels
NeuronTS['Cluster'] = GoodMUAclus

np.save('NeuronTS.npy', NeuronTS)
